<a href="https://colab.research.google.com/github/napoles-uach/DGL_Smell/blob/main/dgl_smell_fruity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install dgl

     |████████████████████████████████| 4.4 MB 4.5 MB/s 


In [2]:
pip install ogb

     |████████████████████████████████| 67 kB 3.6 MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=b3f3d501b0a1549299a750331574904bcca7efac8dc290eb5452af57b5b38364
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built littleutils


In [3]:
pip install rdkit-pypi

     |████████████████████████████████| 18.6 MB 1.3 MB/s 


In [4]:
!wget https://raw.githubusercontent.com/napoles-uach/DGL_Smell/main/train.csv

--2021-08-07 00:01:26--  https://raw.githubusercontent.com/napoles-uach/DGL_Smell/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 197190 (193K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 192.57K  --.-KB/s    in 0.01s   

2021-08-07 00:01:27 (14.5 MB/s) - ‘train.csv’ saved [197190/197190]



In [5]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,SMILES,SENTENCE
0,C/C=C/C(=O)C1CCC(C=C1C)(C)C,"fruity,rose"
1,COC(=O)OC,"fresh,ethereal,fruity"
2,Cc1cc2c([nH]1)cccc2,"resinous,animalic"
3,C1CCCCCCCC(=O)CCCCCCC1,"powdery,musk,animalic"
4,CC(CC(=O)OC1CC2C(C1(C)CC2)(C)C)C,"coniferous,camphor,fruity"


In [6]:
import dgl
from dgl.data import DGLDataset
import torch
import torch as th
import os
from ogb.utils.features import (allowable_features, atom_to_feature_vector,
 bond_to_feature_vector, atom_feature_vector_to_dict, bond_feature_vector_to_dict) 
from rdkit import Chem
import numpy as np

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


In [7]:
def smiles2graph(smiles_string):
    """
    Converts SMILES string to graph Data object
    :input: SMILES string (str)
    :return: graph object
    """

    mol = Chem.MolFromSmiles(smiles_string)

    A = Chem.GetAdjacencyMatrix(mol)
    A = np.asmatrix(A)
    nnodes=len(A)
    nz = np.nonzero(A)
    edge_list=[]
    src=[]
    dst=[]

    for i in range(nz[0].shape[0]):
      src.append(nz[0][i])
      dst.append(nz[1][i])

    u, v = src, dst
    g = dgl.graph((u, v))
    bg=dgl.to_bidirected(g)

    return bg

In [8]:
def feat_vec(smiles_string):
    """
    Returns atom features for a molecule given a smiles string
    """
    # atoms
    mol = Chem.MolFromSmiles(smiles_string)
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom))
    x = np.array(atom_features_list, dtype = np.int64)
    return x

In [9]:
lista_senten=df['SENTENCE'].to_list()
olores=[]
labels=[]
i=0
for olor in lista_senten:
  olor=olor.split(",")
  if 'fruity' in olor:
    labels.append(1)
    olores.append(olor)
  else:
    labels.append(0)
  i+=1
  #olores.append(olor)
# da formato correcto a las sentencias en forma de lista
#olores

In [10]:
# This block makes a list of graphs
lista_mols=df['SMILES'].to_list()

j=0
graphs=[]
execptions=[]
for mol in lista_mols:
  
  g_mol=smiles2graph(mol)

  try:
    g_mol.ndata['feat']=torch.tensor(feat_vec(mol)) 
  except:
    execptions.append(j)
   

  graphs.append(g_mol) #agrega grafo de mol
  j+=1

In [11]:
# Some smiles are not well processed, so they are droped
for i in execptions:
  graphs.pop(i)
  labels.pop(i)

In [12]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
labels=F.one_hot(torch.LongTensor(labels), num_classes=2)

In [14]:
labels

tensor([[0, 1],
        [0, 1],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]])

In [15]:
class SyntheticDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='synthetic')

    def process(self):
        #edges = pd.read_csv('./graph_edges.csv')
        #properties = pd.read_csv('./graph_properties.csv')
        self.graphs = graphs
        self.labels = labels#torch.LongTensor(labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

dataset = SyntheticDataset()
#graph, label = dataset[0]
#print(graph, label)

In [16]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples * 0.8)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size=5, drop_last=False)
test_dataloader = GraphDataLoader(
    dataset, sampler=test_sampler, batch_size=5, drop_last=False)

In [17]:
for batched_graph, labels in train_dataloader:
    print(batched_graph,labels)


Graph(num_nodes=79, num_edges=160,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])
Graph(num_nodes=47, num_edges=90,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([[1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0]])
Graph(num_nodes=73, num_edges=142,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])
Graph(num_nodes=54, num_edges=104,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1]])
Graph(num_nodes=74, num_edges=144,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([[0, 1],
        [1, 0],
        [1

DGLError: ignored

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from dgl.nn import GraphConv
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GraphConv(9, 16) #activation default=None
        self.layer2 = GraphConv(16, 2)    #activation default=None


    def forward(self, g, features):
        x1 = F.relu(self.layer1(g, features)) #ReLU activation function
        x2 = self.layer2(g, x1)
        return x2
net = Net()

In [ ]:
graphs[10].ndata['feat'].float()

In [ ]:

import numpy as np
#g.add_edges(g.nodes(), g.nodes())
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(20):
    #for batched_graph, labels in train_dataloader:
    net.train()
    pred = net(graphs[10], graphs[10].ndata['feat'].float())
        #loss = F.cross_entropy(pred, labels)
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()

#loss_list=[]
#acc_list=[]
#all_logits=[]
#for epoch in range(200):
#    net.train()
#    logits = net(graphs[0], graphs[0].ndata['feat'])
#    pred=sum(logits)
#    logp = F.log_softmax(pred, 0)



    #pred = model(batched_graph, batched_graph.ndata['attr'].float())
    #loss = F.cross_entropy(pred, labels[0])
    
    #print(logits)
    #logp = F.log_softmax(logits, 1)
    #all_logits.append(logp)
#    loss = F.nll_loss(logp[train_mask], labels[train_mask])

#    optimizer.zero_grad()
#    loss.backward()
#    optimizer.step()

#    acc = evaluate(net, g, features, labels, test_mask)
#    loss_list.append(loss.item())
#    acc_list.append(acc)